In [1]:
import pandas as pd 
import cassandra
import re 
import os 
import glob 
import numpy as np 
import json 
import csv 

In [3]:
# Creating a cluser 

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("Connection Established !!")
except Exception as e:
    print(f"Connection Failed !! Error : {e}")

Connection Failed !! Error : ('Unable to connect to any servers', {'127.0.0.1:9042': ConnectionRefusedError(61, "Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused")})


In [4]:
# checking current working directory  
print(f'Current working directory: {os.getcwd()}')

# Get current folder and subfolder event data 
filepath = os.getcwd() + '/event_data'

# Create a list of files and collect each filepath 
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

Current working directory: /Users/hungtran/Documents/Tài liệu/Data Engineer/Data_Modeling_with_Apache_Cassandra


### Processing the files to create the data file csv that will be used for Apache Cassandra tables

In [8]:
# initiating an empty list of rows that will be generated from each file 
full_data_row_list = []

# for every filepath in the file path list 
for f in file_path_list: 
    with open(f, 'r', encoding='utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile)
        next(csvreader)

    # extracting each data row one by one and append it 
        for line in csvreader:
            full_data_row_list.append(line)

print(f"Total row: {len(full_data_row_list)}")
print(f"Sample data: {full_data_row_list[:5]}")

# creating s smaller event data csv file called event_datafile_full csv that will be used to insert data into the Cassandra table 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_row_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

Total row: 8056
Sample data: [['Harmonia', 'Logged In', 'Ryan', 'M', '0', 'Smith', '655.77751', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'Sehr kosmisch', '200', '1.54224E+12', '26'], ['The Prodigy', 'Logged In', 'Ryan', 'M', '1', 'Smith', '260.07465', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'The Big Gundown', '200', '1.54224E+12', '26'], ['Train', 'Logged In', 'Ryan', 'M', '2', 'Smith', '205.45261', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'Marry Me', '200', '1.54224E+12', '26'], ['', 'Logged In', 'Wyatt', 'M', '0', 'Scott', '', 'free', 'Eureka-Arcata-Fortuna, CA', 'GET', 'Home', '1.54087E+12', '563', '', '200', '1.54225E+12', '9'], ['', 'Logged In', 'Austin', 'M', '0', 'Rosales', '', 'free', 'New York-Newark-Jersey City, NY-NJ-PA', 'GET', 'Home', '1.54106E+12', '521', '', '200', '1.54225E+12', '12']]


In [ ]:
# Creating Keyspace 

keyspace_query = """    CREATE KEYSPACE IF NOT EXISTS sparkify
                        with REPLICATION =  {'class': 'SimpleStrategy', 'replication_factor' : 1} """

# Creating Keyspace 
try: 
    session.execute(keyspace_query)
except Exception as e:
    print(f'Failed to create keyspace!! Error : {e}')

In [ ]:
# Setting Keyspace 
session.set_keyspace('sparkify')

In [ ]:
# Create table for query
create_query1 = """
                CREATE TABLE IF NOT EXISTS session_item (
                    artist text, 
                    song text, 
                    length float, 
                    sessionId int,
                    itemInSession text,
                    PRIMARY KEY (sessionID, (itemInSessionID))
                )
                """

try:
    session.execute(create_query1)
except Exception as e: 
    print(f'Table creation failed!! Error: {e}')

In [ ]:
# Using the event file 
file = 'event_datafile_new.csv'

# Reading csv file and inserting rows into cassandra tables
with open(file, encoding = 'utf8') as f: 
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader: 
        query = "INSERT INTO session_item (artist, song, length, sessionId, itemInSession) VALUES (%s,%s,%s,%s,%s)"
        session.execute(query,(line[0], line[10], float(line[5]), int(line[8]), int(line[3])) )

In [ ]:
# SELECT statement to verify that the data have been inserted into each table
select_query1 = "SELECT artist, song, length FROM session_item where sessionID = 338 and itemInSession = 4"
try: 
    rows = session.execute(select_query1)
except Exception as e: 
    print(e)

for row in rows:
    print(row)

In [ ]:
# Creating table for query2
create_query2 = """
                CREATE TABLE IF NOT EXISTS user_session(
                    sessionID int, 
                    userId int,
                    artist text, 
                    song text, 
                    firstName text, 
                    lastName text,
                    PRIMARY KEy ((sessionId, userID), itemInSession)
                )
                """
try:
    session.execute(create_query1)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csvreader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_session (sessionId, userId, artist, song, firstName, lastName, itemInSession)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[10]), line[0], line[9], line[1], line[4], int(line[3])))

In [ ]:
# SELECT statement to verify that the data was entered into the table 
select_query2 = "SELECT artist, song, firstName, lastName FROM user_session where sessionId = 182 and userId = 1"
try:
    rows = session.execute(select_query2)
except Exception as e: 
    print(e)

In [ ]:
# Drop the tables before closing the session
session.execute("DROP TABLE IF EXISTS sparkify.session_item")
session.execute("DROP TABLE IF EXISTS sparkify.user_session")